In [ ]:
import geopandas as gpd
import pandas as pd
import requests
import math
import numpy
from datetime import datetime
from shapely import wkt
from shapely.errors import WKTReadingError
from shapely.geometry import mapping, shape
from shapely import Polygon
import warnings
import os
warnings.filterwarnings("ignore")

In [ ]:
import json

def parse_json_safe(x):
    if isinstance(x, dict):
        return x
    try:
        return json.loads(x)
    except (json.JSONDecodeError, TypeError):
        return {}

In [ ]:
login = {
    "username":"Dorito",
    "password":"Portador123"
}
# Crear una sesión para mantener la cookie
session = requests.Session()
response = session.post("https://produccion.local/api/user/login/", login, verify=False)
def get_data_paginated(url):
    login = {
        "username":"Dorito",
        "password":"Portador123"
    }
    # Crear una sesión para mantener la cookie
    session = requests.Session()
    response = session.post("https://produccion.local/api/user/login/", login, verify=False)
    datos = []
    while url:
        response = session.get(url, verify=False)
        info = response.json()
        datos.extend(info["results"])
        url = info["next"]
    return datos

def get_data_not_paginated(url):
    login = {
        "username":"Dorito",
        "password":"Portador123"
    }
    # Crear una sesión para mantener la cookie
    session = requests.Session()
    response = session.post("https://produccion.local/api/user/login/", login, verify=False)
    response = session.get(url, verify=False)
    info = response.json()
    return info

In [ ]:
url_productor = "https://produccion.local/api/v1/productor/"
url_productor_data = "https://produccion.local/api/v1/productor/mongo_get/productor_data/"
print("cargando datos productor")
datos_productor = get_data_not_paginated(url_productor)
print("cargando datos productor")
datos_productor_data = get_data_not_paginated(url_productor_data)
df_productor = pd.json_normalize(datos_productor)
df_productor_data = pd.json_normalize(datos_productor_data)
df_productor = df_productor.merge(
    df_productor_data,
    left_on="productor_data",
    right_on="_id",
    how="left"
)
df_productor.to_excel("./productores.xlsx")

In [ ]:
# Obtener y normalizar datos
url = "https://produccion.local/api/v1/poligonos_lotes/"
url_productor = "https://produccion.local/api/v1/productor/"
url_productor_data = "https://produccion.local/api/v1/productor/mongo_get/productor_data/"
url_fincas = "https://produccion.local/api/v1/poligonos_fincas/"
url_municipio = "https://produccion.local/api/v1/municipio/"
url_departamento = "https://produccion.local/api/v1/vereda/"
url_lotes = "https://produccion.local/api/v1/poligonos_lotes/mongo_get/mongo_atribute/"
print("cargando datos lotes")
datos = get_data_paginated(url)
print("cargando datos productor")
datos_productor = get_data_not_paginated(url_productor)
print("cargando datos productor")
datos_productor_data = get_data_not_paginated(url_productor_data)
print("cargando datos fincas")
datos_fincas = get_data_paginated(url_fincas)
print("cargando datos lotes")
datos_lotes = get_data_not_paginated(url_lotes)
print("Cargando datos municipios")
datos_municipios = get_data_not_paginated(url_municipio)
print("cargando datos departamentos")
datos_departamentos = get_data_not_paginated(url_departamento)



df = pd.json_normalize(datos)
df_productor = pd.json_normalize(datos_productor)
df_productor_data = pd.json_normalize(datos_productor_data)
df_fincas = pd.json_normalize(datos_fincas)
df_lotes = pd.json_normalize(datos_lotes)
df_municipio = pd.json_normalize(datos_municipios)
df_departamento = pd.json_normalize(datos_departamentos)


In [ ]:
# Filtrar productores que tienen postgres_data.id
df_productor = df_productor[df_productor["id"].notna()]
# Realizar los merge sin eliminar filas del DataFrame principal
df_completo = df.merge(
    df_productor,
    left_on="productor",
    right_on="id",
    how="left"
)

df_completo = df_completo.merge(
    df_productor_data,
    left_on="productor_data",
    right_on="_id",
    how="left"
)

df_completo = df_completo.merge(
    df_fincas,
    left_on="finca",
    right_on="id",
    how="left"
)

df_completo = df_completo.merge(
    df_lotes,
    left_on="id_x",
    right_on="poligono_id",
    how="left"
)

# Ejemplo: renombrar id en municipio y departamento
df_municipio = df_municipio.rename(columns={"id": "id_municipio"})
df_departamento = df_departamento.rename(columns={"id": "id_departamento"})
df_departamento = df_departamento.rename(columns={"municipio": "alkdsjflkajsñldkfjasdl"})


df_completo = df_completo.merge(
    df_municipio,
    left_on="municipio_x",
    right_on="id_municipio",
    how="left"
)

df_completo = df_completo.merge(
    df_departamento,
    left_on="vereda_x",
    right_on="id_departamento",
    how="left"
)



In [ ]:
df_completo.columns

In [ ]:
# Construir el GeoDataFrame (con geometrías inválidas como None)

def parse_geom_safe(wkt_string):
    try:
        if isinstance(wkt_string, str) and "SRID=" in wkt_string:
            return wkt.loads(wkt_string.split(";", 1)[1])
    except:
        pass
    return None  # simplemente ignora lo inválido

# Aplicar solo a las válidas
df_completo["geometry"] = df_completo["poligono_x"].apply(parse_geom_safe)

# Crear el GeoDataFrame sin que explote
gdf = gpd.GeoDataFrame(df_completo, geometry="geometry", crs="EPSG:4326")

# Validación de cantidad
# assert len(gdf) == 10530, f"Se esperaban 10530 registros, pero hay {len(gdf)}"

In [ ]:
from shapely import unary_union
lista_de_necesarios = [83239477, 55056272, 1082155195]
respuesta = []
for documento in lista_de_necesarios:
    gdf_filtro = gdf.loc[gdf["postgres_data.documento"] == documento]
    poligono = unary_union(gdf_filtro.geometry)
    centroide = poligono.centroid
    respuesta.append({
        "documento": documento,
        "nombre": gdf_filtro["datos.nombre_completo"].iloc[0],
        "codigo_productor": gdf_filtro["codigo_productor"].iloc[0],
        "codigo_finca": gdf_filtro["codigo_finca"].iloc[0],
        "latitud": centroide.y,
        "longitud": centroide.x,
        "geometry": poligono
    })
gdf_final = gpd.GeoDataFrame(pd.DataFrame(respuesta), geometry="geometry", crs=4326)
gdf_final.to_file("./Poligonos_Faltante_CP_12_DICIEMBRE.geojson")

In [ ]:
gdf["datos.produccion"].unique()

In [ ]:
gdf["datos.categoria"] = gdf["datos.categoria"].fillna('Qfield')
gdf["datos.produccion"] = gdf["datos.produccion"].fillna(0)
gdf["datos.produccion"] = gdf["datos.produccion"].isin([1, "Produccion", "1"])

In [ ]:
gdf.columns

In [ ]:
gdf["datos.categoria"].unique()

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from shapely.validation import make_valid

def filtrar_poligonos_superpuestos(gdf, umbral=0.85):
    # Asegurar que la fecha sea datetime
    gdf = gdf.copy()
    gdf["fecha"] = pd.to_datetime(gdf["fecha"], format="%d/%m/%Y %H:%M:%S", errors="coerce")

    # Ordenar por fecha descendente (primero los más recientes)
    gdf = gdf.sort_values(by="fecha", ascending=False).reset_index(drop=True)

    # Lista de índices que queremos conservar
    indices_validos = []

    for i, row_i in gdf.iterrows():
        geom_i = row_i.geometry
        area_i = geom_i.area
        keep = True

        for j in indices_validos:
            geom_j = gdf.loc[j, "geometry"]
            area_j = geom_j.area

            inter = geom_i.intersection(geom_j).area
            if inter > 0:
                # calcular porcentaje de solapamiento respecto al polígono más pequeño
                overlap = inter / min(area_i, area_j)
                if overlap >= umbral:
                    keep = False
                    break

        if keep:
            indices_validos.append(i)

    return gdf.loc[indices_validos].reset_index(drop=True)
def filtrar_poligonos_priorizar_pequenos(gdf, umbral=0.01):
    gdf = gdf.copy()

    # Reparar geometrías
    gdf["geometry"] = gdf["geometry"].apply(lambda g: make_valid(g))

    # Calcular áreas
    gdf["area"] = gdf.geometry.area

    # Pequeños primero
    gdf = gdf.sort_values("area", ascending=True).reset_index(drop=True)

    indices_validos = []

    for i, row_i in gdf.iterrows():
        geom_i = row_i.geometry
        area_i = row_i.area

        keep = True
        
        # Comparar con los ya aceptados (que son pequeños o iguales)
        for j in indices_validos:
            geom_j = gdf.loc[j, "geometry"]
            area_j = gdf.loc[j, "area"]

            # No necesitamos verificar el caso contrario: j siempre es pequeño o igual
            inter = geom_i.intersection(geom_j).area
            if inter > 0:
                overlap = inter / min(area_i, area_j)
                if overlap >= umbral:
                    # geom_i (que es más grande o igual) se elimina
                    keep = False
                    break
        
        if keep:
            indices_validos.append(i)

    return gdf.loc[indices_validos].reset_index(drop=True)



In [ ]:
gdf['datos.fecha_actividad'] = pd.to_datetime(gdf['datos.fecha_actividad'], errors='coerce')


In [ ]:
gdf["datos.gramos_plantas"] = gdf["datos.gramos_plantas"].fillna(500)

In [ ]:
gdf["datos.numero_plantas"] = pd.to_numeric(gdf["datos.numero_plantas"], errors="coerce")
gdf["datos.gramos_plantas"] = pd.to_numeric(gdf["datos.gramos_plantas"], errors="coerce")

gdf["datos.kg_produccion"] = gdf["datos.numero_plantas"] * (gdf["datos.gramos_plantas"] / 1000)


In [ ]:
# # Modelo Basado en Municipios

# import geopandas as gpd
# import pandas as pd

# # Crear un DataFrame vacío para almacenar los resultados
# resultados = []
# def calcular_reportes(gdf):
#     # asegurar conversión
#     gdf["datos.numero_plantas"] = pd.to_numeric(gdf["datos.numero_plantas"], errors="coerce")
#     gdf["area_x"] = pd.to_numeric(gdf["area_x"], errors="coerce")

#     respuesta = {}
#     respuesta["municipio"] = gdf["municipio"].iloc[0] if not gdf.empty else ""
#     respuesta["departamento"] = gdf["departamento"].iloc[0] if not gdf.empty else ""
    
#     # for categoria in gdf["datos.categoria"].unique():   # <- mejor unique()
#     #     gdf_categoria = gdf.loc[gdf["datos.categoria"] == categoria]
#     #     gdf_categoria_produccion = gdf_categoria.loc[gdf_categoria["datos.produccion"]]
#     #     gdf_categoria_levante = gdf_categoria.loc[~gdf_categoria["datos.produccion"]]
#     #     respuesta[f"{categoria}_area_levante"] = gdf_categoria_levante["area_x"].sum()
#     #     respuesta[f"{categoria}_plantas_levante"] = gdf_categoria_levante["datos.numero_plantas"].sum()
#     #     respuesta[f"{categoria}_area_produccion"] = gdf_categoria_produccion["area_x"].sum()
#     #     respuesta[f"{categoria}_plantas_produccion"] = gdf_categoria_produccion["datos.numero_plantas"].sum()
    
#     gdf_categoria_final = gdf
#     gdf_categoria_produccion = gdf_categoria_final.loc[gdf_categoria_final["datos.produccion"]]
#     gdf_categoria_levante = gdf_categoria_final.loc[~gdf_categoria_final["datos.produccion"]]
#     respuesta["area_levante"] = gdf_categoria_levante["area_x"].sum()
#     respuesta["plantas_levante"] = gdf_categoria_levante["datos.numero_plantas"].sum()
#     respuesta["area_produccion"] = gdf_categoria_produccion["area_x"].sum()
#     respuesta["plantas_produccion"] = gdf_categoria_produccion["datos.numero_plantas"].sum()
#     respuesta["fecha_actividad_promedio"] = gdf_categoria_produccion["datos.fecha_actividad"].mean()
#     respuesta["densidad_promedio"] = gdf_categoria_final["datos.densidad"].mean()
#     respuesta["kg_produccion"] = gdf_categoria_produccion["datos.kg_produccion"].sum()
#     respuesta["kg_levante"] = gdf_categoria_levante["datos.kg_produccion"].sum()

#     return respuesta


# # gdf = filtrar_poligonos_superpuestos(gdf, umbral=0.85)

# for documento in gdf["municipio"].unique():
#     print(documento)
#     gdf_productor = gdf.loc[gdf["municipio"] == documento]
#     gdf_productor = gdf_productor.loc[~gdf_productor["geometry"].isna()]
#     resultados.append(calcular_reportes(gdf_productor))

In [ ]:
import geopandas as gpd
import pandas as pd

# Crear un DataFrame vacío para almacenar los resultados
resultados = []
def calcular_reportes(gdf):
    # asegurar conversión
    gdf["datos.numero_plantas"] = pd.to_numeric(gdf["datos.numero_plantas"], errors="coerce")
    gdf["area_x"] = pd.to_numeric(gdf["area_x"], errors="coerce")

    respuesta = {}
    respuesta["documento"] = gdf["documento"].iloc[0] if not gdf.empty else ""
    respuesta["nombre"] = gdf["datos.nombre_completo"].iloc[0] if not gdf.empty else ""
    respuesta["codigo_productor"] = gdf["codigo_productor"].iloc[0] if not gdf.empty else ""
    
    # for categoria in gdf["datos.categoria"].unique():   # <- mejor unique()
    #     gdf_categoria = gdf.loc[gdf["datos.categoria"] == categoria]
    #     gdf_categoria_produccion = gdf_categoria.loc[gdf_categoria["datos.produccion"]]
    #     gdf_categoria_levante = gdf_categoria.loc[~gdf_categoria["datos.produccion"]]
    #     respuesta[f"{categoria}_area_levante"] = gdf_categoria_levante["area_x"].sum()
    #     respuesta[f"{categoria}_plantas_levante"] = gdf_categoria_levante["datos.numero_plantas"].sum()
    #     respuesta[f"{categoria}_area_produccion"] = gdf_categoria_produccion["area_x"].sum()
    #     respuesta[f"{categoria}_plantas_produccion"] = gdf_categoria_produccion["datos.numero_plantas"].sum()
    
    gdf_categoria_final = gdf
    gdf_categoria_produccion = gdf_categoria_final.loc[gdf_categoria_final["datos.produccion"]]
    gdf_categoria_levante = gdf_categoria_final.loc[~gdf_categoria_final["datos.produccion"]]
    respuesta["area_levante"] = gdf_categoria_levante["area_x"].sum()
    respuesta["plantas_levante"] = gdf_categoria_levante["datos.numero_plantas"].sum()
    respuesta["area_produccion"] = gdf_categoria_produccion["area_x"].sum()
    respuesta["plantas_produccion"] = gdf_categoria_produccion["datos.numero_plantas"].sum()
    respuesta["densidad_promedio"] = gdf_categoria_produccion["datos.densidad"].mean()
    respuesta["fecha_actividad_promedio"] = gdf_categoria_produccion["datos.fecha_actividad"].mean()
    return respuesta



for documento in gdf["documento"].unique():
    gdf_productor = gdf.loc[gdf["documento"] == documento]
    gdf_productor = gdf_productor.loc[~gdf_productor["geometry"].isna()]
    gdf_limpio = filtrar_poligonos_superpuestos(gdf_productor, umbral=0.85)
    resultados.append(calcular_reportes(gdf_limpio))

In [ ]:
df = pd.DataFrame(resultados)

In [ ]:
# ids = [17615621, 1077868357, 4888454, 4939216, 14191767, 83058202, 4911887, 4946046, 83239477, 83055844, 4633378, 12188324, 12190707, 83181887, 83183128, 4939015, 1081517021, 12186723, 1081515806, 93348205, 4939337, 4925846, 10539656, 83240125, 12192355, 12205276, 1080932294, 26575053, 14256994, 79397128, 1080361312, 83056611, 55056272, 79749725, 55059937, 12192720, 12208197, 14212934, 83165054, 26541132, 12365043, 83182724, 4939287, 83058867, 1126001183, 1080364946, 1077873948, 4940702, 83258281, 1080360916, 1082127141, 4900850, 4881181, 83058107, 12196217, 1080182506, 1110538441, 26575324, 1080363194, 1077848393, 12196895, 1082126268, 12202435, 12188249]
# df = df.loc[~df["documento"].isin(ids)]

<h3>Ahora vamos con las fincas</h3>

In [ ]:
# Obtener y normalizar datos
url = "https://produccion.local/api/v1/poligonos_fincas/"
url_productor = "https://produccion.local/api/v1/productor/"

print("cargando datos fincas")
datos = get_data_paginated(url)
print("cargando datos productor")
datos_productor = get_data_not_paginated(url_productor)

df_2 = pd.json_normalize(datos)
df_productor = pd.json_normalize(datos_productor)


In [ ]:
# Filtrar productores que tienen postgres_data.id
df_productor = df_productor[df_productor["id"].notna()]
# Realizar los merge sin eliminar filas del DataFrame principal
df_completo = df_2.merge(
    df_productor,
    left_on="productor",
    right_on="id",
    how="left"
)
df_completo = df_completo.merge(
    df_municipio,
    left_on="municipio",
    right_on="id_municipio",
    how="left"
)

In [ ]:
# Construir el GeoDataFrame (con geometrías inválidas como None)

def parse_geom_safe(wkt_string):
    try:
        if isinstance(wkt_string, str) and "SRID=" in wkt_string:
            return wkt.loads(wkt_string.split(";", 1)[1])
    except:
        pass
    return None  # simplemente ignora lo inválido

# Aplicar solo a las válidas
df_completo["geometry"] = df_completo["poligono"].apply(parse_geom_safe)

# Crear el GeoDataFrame sin que explote
gdf = gpd.GeoDataFrame(df_completo, geometry="geometry", crs="EPSG:4326")

In [ ]:
gdf.columns

In [ ]:
# import pandas as pd
# import geopandas as gpd
# from shapely.ops import unary_union
# from shapely.validation import make_valid

# # Crear columnas nuevas
# df["Area_Finca"] = None
# df["codigo_finca_corregido"] = None
# errores = []

# for i, row in df.iterrows():
#     print(f"{i+1}/{len(df)}")
#     municipio = str(row["municipio"]).strip()

#     # Filtrar el gdf por municipio
#     gdf_filtro = gdf.loc[gdf["municipio_y"] == municipio].copy()
#     if gdf_filtro.empty:
#         errores.append((municipio, "Sin geometrías"))
#         continue

#     # Reproyectar
#     gdf_filtro = gdf_filtro.to_crs(3116)

#     # --- Validar y corregir geometrías ---
#     gdf_filtro["geometry"] = gdf_filtro["geometry"].apply(
#         lambda geom: make_valid(geom) if geom and not geom.is_valid else geom
#     )

#     # Algunos "make_valid" devuelven colecciones, extrae solo polígonos válidos
#     gdf_filtro["geometry"] = gdf_filtro["geometry"].apply(
#         lambda geom: geom.geoms[0] if hasattr(geom, "geoms") else geom
#     )

#     # --- Calcular área ---
#     try:
#         poligono = unary_union(gdf_filtro.geometry)
#         area_ha = poligono.area / 10000  # a hectáreas
#         df.at[i, "Area_Finca"] = area_ha
#     except Exception as e:
#         errores.append((municipio, str(e)))
#         print(f"⚠️ Error en {municipio}: {e}")


In [ ]:
df.to_excel("./analisis_por_municipio.xlsx")

In [ ]:
import pandas as pd
import re
from shapely.ops import unary_union

# Creamos nuevas columnas
df["Area Finca"] = None
df["codigo_finca_corregido"] = None
errores = []

for i, row in df.iterrows():
    print(f"{i+1}/{len(df)}")
    documento = str(row["documento"]).strip()

    # Filtrar el gdf por documento
    gdf_filtro = gdf.loc[gdf["documento"] == row["documento"]]

    if not gdf_filtro.empty:
        codigo_fincas = str(gdf_filtro["codigo_finca"].unique()[0]).strip()

        # --- Validación del código ---
        # Extraer 3 últimos dígitos del documento
        ultimos_doc = documento[-3:] if documento.isdigit() else None

        # Buscar si el cuarto, tercer y segundo carácter desde el final son números
        sufijo = codigo_fincas[-7:-4]  # son los XXX si existen
        try:
            sufijo = int(sufijo)
        except:
            sufijo = "ERROR"
        if sufijo == "ERROR":
            # El código es inválido → corregir
            ultimos_4 = codigo_fincas[-4:]
            prefijo = codigo_fincas[:-4]
            codigo_corregido = prefijo+str(int(row["documento"]))[-3:]+ultimos_4
            df.at[i, "codigo_finca_corregido"] = codigo_corregido
            errores.append({
                "indice": i,
                "documento": documento,
                "codigo_original": codigo_fincas,
                "codigo_corregido": codigo_corregido
            })
        else:
            # Código válido, lo dejamos igual
            df.at[i, "codigo_finca_corregido"] = codigo_fincas

        # --- Calcular área ---
        gdf_filtro = gdf_filtro.to_crs(3116)
        poligono = unary_union(gdf_filtro.geometry)
        area_ha = poligono.area / 10000
        df.at[i, "Area Finca"] = area_ha

    else:
        # Caso donde no hay finca en gdf
        codigo_productor = row["codigo_productor"]
        ultimos_doc = documento[-3:] if documento.isdigit() else "000"
        codigo_fincas = codigo_productor.replace("PR", "FN") + ultimos_doc + "QFL1"

        df.at[i, "codigo_finca_corregido"] = codigo_fincas
        df.at[i, "Area Finca"] = 0

# Mostrar resumen de casos erróneos
df_errores = pd.DataFrame(errores)



In [ ]:

fecha_hoy = datetime.now()
fecha_hoy_string = fecha_hoy.strftime("%d%m%Y")
# print(fecha_hoy_string)
df.to_excel(f"./Inf_Poligonal_{fecha_hoy_string}.xlsx")

In [ ]:
# df_ibarra = df.loc[df["documento"].isin([26441476, 26491773, 12208646, 31922655, 83226660, 26575321, 36294651, 83166252, 83212381, 1082124955, 1077856601, 1083924601, 1006093686, 1081515494])]

In [ ]:
# df_ibarra.to_excel("./aso_nuevos_resultado_2.xlsx")

In [ ]:
# gdf.columns

In [ ]:
# gdf["datos.variedad"].unique()